In [ ]:
# Dependencies
from census import Census
from config import census_key
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt


# Census API Key
c = Census(census_key)

In [ ]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E"), {'for': 'state:*'})
eduVariable = ["B01003_001E", "B17001_002E"]
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

census_pd = census_pd.astype('int32')
census_pd.head()

In [ ]:
state_codes = pd.read_csv("../Resources/state_codes.csv")
state_codes.head()

In [ ]:
censusData = pd.merge(census_pd, state_codes, on = 'state')
censusData = censusData[['State Name', 'B01003_001E', 'B17001_002E']]
censusData = censusData.rename(columns = {"B01003_001E":'Variable Name 1',
                                          "B17001_002E":'Variable Name 2'})
censusData.head()